<a href="https://colab.research.google.com/github/theaveas/ComputationalDesign-for-Architect/blob/main/Pix2PixHD_Next_Frame_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Next Frame Prediction using Pix2PixHD

By Derrick Schultz

Forked repo and tutorial based on [JC Testud’s excellent repo and Medium](https://medium.com/@jctestud/video-generation-with-pix2pix-aed5b1b69f57) article.

## Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Check to see what GPU we’re assigned

In [ ]:
!nvidia-smi -L

## Install libraries and dependencies



In [ ]:
import os
if os.path.isdir("/content/drive/MyDrive/nfp-colab/pix2pixHD/"):
    %cd /content/drive/MyDrive/nfp-colab/pix2pixHD/
    # !git pull
    !pip install dominate
else:
    %cd /content/drive/MyDrive
    !mkdir nfp-colab
    %cd nfp-colab
    !git clone -b video https://github.com/dvschultz/pix2pixHD.git
    !pip install dominate
    %cd pix2pixHD/

## Extract frames from video

Upload a video to either Colab or Google Drive.

* `-video` is the path to the video file
* `-name` should be a unique name (think of it like a project name)
* `-width` and `-height` **must** to be a multiple of 32
* `-p2pdir` leave as `.` unless you know it shouldn’t be that ;)


In [ ]:
!python extract_frames.py -video /content/cuttlefish1.mov -name cuttlefish1 -p2pdir . -width 1280 -height 736

## Train your model

### Initial training

Note: if you have a large dataset, this may timeout initially (`ValueError: __len__() should return >= 0`). Give it a minute or two and run it again.

*   `--name` should be a unique name (think of it like a project name). For your sanity I recommend using the same `-name` as above.
*   `--dataroot` should point to your dataset. This will usually be in `./datasets/[your project name]`

 



In [ ]:
!python train_video.py --name cuttlefish1 --dataroot ./datasets/cuttlefish1/ --save_epoch_freq 5 #--continue_train

### Continue Training
To pick up from a previous training session run the same command you ran to start with and append `--continue_train` to the end of the command.

In [ ]:
!python train_video.py --name cuttlefish1 --dataroot ./datasets/cuttlefish1/ --save_epoch_freq 1 --continue_train

#Generating Videos

To generate a video from your completed model, run the `generate_video.py` script. I recommend keeping the `--name` and `--dataroot` arguments the same.

There are a number of additional arguments you’ll need to include in this command:


*   `--fps` frame rate for your video
*   `--how_many` how many frames you want to create (this + fps = video length)
*   `--which_epoch` which epoch you want to generate videos from (note: if you choose `133` but there’s no epoch 133 model file, this will throw an error)
*   `--start_from` by default your video will start predicting images from the 60th frame of your training set. You can pass in the path to a different frame to have it start from that frame

I recommend playing with both the `--which_epoch` and `--start_from` arguments as you can get dramatically different results by changing in the inputs here.





In [ ]:
!python generate_video.py --name cuttlefish1 --dataroot ./datasets/cuttlefish1/ --fps 24 --how_many 600 --which_epoch latest 

In [ ]:
import os

def processFolder(folder, epoch = 10, frameCount = 240, skipCount = 1):
  files = os.listdir(folder)

  count = 0
  for f in files:
    
    if (count % skipCount == 0):
      print(f)
      if epoch == 'latest':
        command = 'python generate_video.py --name glitch_white_circle --dataroot ./datasets/glitch-circle-white_dataset/ --fps 60 --how_many %i --which_epoch latest --start_from %s/%s' % ( frameCount, folder, f)
      else:
        command = 'python generate_video.py --name glitch_white_circle --dataroot ./datasets/glitch-circle-white_dataset/ --fps 24 --how_many %i --which_epoch %i --start_from %s/%s' % ( frameCount, epoch, folder, f)
      os.system(command)
    count += 1

processFolder('./datasets/fuck-white/train_frames',1,600,3050)